# Noisy LLM Experimental Notebook
@author: sjkro1

## Developer Environment
The following package versions are what were used to develop this notebook.

python==3.12.7
cuda==11.5
pytorch==2.4.1
transformers==4.45.2
accelerate==1.0.0

## Environment Setup

In [1]:
# import dependencies
import torch
import accelerate
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [2]:
# retrieve GPU device
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch.set_default_device("cuda")

In [3]:
# Add Gaussian noise to the embeddings @author: chatgpt + sjkro1
def add_gaussian_noise(tensor, mean=0.0, std=0.1, dtype=torch.float16):
    noise = torch.randn(tensor.size(),dtype=dtype).to(tensor.device) * std + mean
    return tensor + noise

# Create a wrapper around the embedding layer to add noise
class NoisyEmbedding(torch.nn.Module):
    def __init__(self, original_embedding, mean, std, dtype=torch.float16):
        super(NoisyEmbedding, self).__init__()
        self.original_embedding = original_embedding
        self.mean = mean
        self.std = std
        self.dtype = dtype
    
    def forward(self, input_ids):
        embeddings = self.original_embedding(input_ids)
        noisy_embeddings = add_gaussian_noise(embeddings, self.mean, self.std, self.dtype)
        return noisy_embeddings

## Phi 2 Embedding Noise

In [4]:
# load models on GPU
phi2 = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True).to(device)
phi2_tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
# model input
prompt = 'Who is god?'
inputs = phi2_tokenizer(prompt, return_tensors="pt").to(device)

# noise paramaeters - for no noise set both to zero
mean = 0.0
std = 0.1

In [7]:
# retrieve model embedding layer and replace with noisy embedding layer
phi2.eval()
embedding = phi2.get_input_embeddings()
embedding(inputs['input_ids'])
noisy_embedding = NoisyEmbedding(embedding, mean, std)
phi2.set_input_embeddings(noisy_embedding)

# generate outputs
outputs = phi2.generate(**inputs, max_length=100, pad_token_id=phi2_tokenizer.eos_token_id)
text = phi2_tokenizer.batch_decode(outputs)[0]
print(text)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Who is god?..
otenation of.
 on nity

 dextodayineoano
ineoan ooan oan oan toan oanoityonan yon etanetan tonetan etan etan etan etan etan et ets etan fan etan et ran et etan etan etan et etan etan et et et et et et rena et et et et et


## Phi 3 Embedding Noise

In [11]:
phi3 = AutoModelForCausalLM.from_pretrained( 
    "microsoft/Phi-3-mini-128k-instruct",  
    device_map="cuda",  
    torch_dtype="auto",  
    trust_remote_code=True,  
) 

phi3_tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct") 

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [148]:
# model input
system_context = "You are a helpful AI Philosopher."
user_prompt = "Happy"

messages = [ 
    {"role": "system", "content": system_context}, 
    {"role": "user", "content": user_prompt}
] 

# noise paramaeters - for no noise set both to zero
mean = 0.0
std = 0.1

In [24]:
# TODO: update embedding layer without reloading model

phi3 = AutoModelForCausalLM.from_pretrained( 
    "microsoft/Phi-3-mini-128k-instruct",  
    device_map="cuda",  
    torch_dtype="auto",  
    trust_remote_code=True,  
) 

phi3_tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct") 

# model embedding setup
phi3.eval()
embedding = phi3.get_input_embeddings()
noisy_embedding = NoisyEmbedding(embedding, mean, std, dtype=torch.bfloat16)
phi3.set_input_embeddings(noisy_embedding)

pipe = pipeline( 
    "text-generation", 
    model=phi3, 
    tokenizer=phi3_tokenizer, 
) 

generation_args = { 
    "max_new_tokens": 500, 
    "return_full_text": False, 
    "temperature": 0.0, 
    "do_sample": False, 
} 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [25]:
# model input
phi3.get_input_embeddings()

NoisyEmbedding(
  (original_embedding): Embedding(32064, 3072, padding_idx=32000)
)

In [26]:
output = pipe(messages, **generation_args) 
print(output[0]['generated_text']) 

 with to ai Ai Philiapper. The are about to be Aipermenter to A A The is about to be A Ai Philoever. Ai A A A A A A A A A 
 
 �  A   � 1 to be to to to to 
 
 

 
 
 to to  to  to  to A to A A  A 


 1  

  to A 



    
 
 A 
  
  

 

  
    
  
 
  
 ist  to be  
 

  
        to bebe  

 

 1  to to  
 


  to 
    
   
 
  
 
 
    
        
        >    
   
  �  
                
    
          
    
       
                
  
   
     
 
 
           
          
  
  
            
    �    
 
  
        >   
       
 
   �   
        A           >  
  �  �   �    >   ��   > 
      �    
  
 �      >     
 
         


## Vector Rotation

In [4]:
import numpy as np
from sklearn.preprocessing import normalize

In [5]:
def cos_sim(a, b): return np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
def proj(a, b): return b * np.dot(a, b) / np.dot(b, b)
def scalar_proj(a, b): return np.dot(a, b) / np.dot(b, b)

In [8]:
# https://stackoverflow.com/questions/33658620/generating-two-orthogonal-vectors-that-are-orthogonal-to-a-particular-direction
# single vector test
costheta = 0.95
v = np.random.randn(5)
u = v / np.linalg.norm(v)
r = np.random.multivariate_normal(np.zeros_like(v), np.eye(len(v)))
# Form a vector perpendicular to v:
uperp = r - r.dot(u)*u

# Make it a unit vector:
uperp = uperp / np.linalg.norm(uperp)

# w is the linear combination of u and uperp with coefficients costheta
# and sin(theta) = sqrt(1 - costheta**2), respectively:
w = costheta*u + np.sqrt(1 - costheta**2)*uperp


In [10]:
# multiple vector
V = np.random.randn(5, 5)
r = np.random.multivariate_normal(np.zeros(5), np.eye(5))
costheta = 0.95

U = V / np.linalg.norm(V, axis=1)[:,None]

# form perpendicular vectors
U_perp = r - U*np.dot(U, r)[:, None]

# make unit vectors
U_perp = U_perp / np.linalg.norm(U_perp, axis=1)[:,None]

# linear combination of vectors
W = costheta*U + np.sqrt(1 - costheta**2)*U_perp

# project vectors - chatgpt help for numpy syntax for vector math
W_proj = W*((np.sum(V*W, axis=1) / np.sum(W*W, axis=1))[:, None])

print(cos_sim(W_proj[0], V[0]))

[[-0.38949619 -0.72220237  0.9478041   0.96031776 -1.25666709]
 [ 1.02377275  0.43160545  1.16713001  0.38004066  0.3514239 ]
 [-0.05732139  0.76318127  0.4792831  -2.2026694  -0.338954  ]
 [ 0.24259393 -0.22796365 -0.33039744  0.06482378 -1.41673122]
 [-0.62447245  0.61634633 -0.7954119   0.31153397 -0.22941895]]
[ 0.46417992 -0.5982623  -1.19704227 -0.91910295 -0.50384688]
0.95


In [122]:
def rotate_vectors(V, costheta):

    r = torch.distributions.multivariate_normal.MultivariateNormal(torch.zeros(V.shape[2]), torch.diag(torch.ones(V.shape[2]))).rsample().type(torch.bfloat16)
    V = V.squeeze(dim=0)
    # normalise vector

    U = V / torch.linalg.norm(V, axis=1)[:,None]

    # form perpendivular vectors
    U_perp = r - U*torch.matmul(U, r)[:, None]

    # make perpendicular vectors unit vectors
    U_perp = U_perp / torch.linalg.norm(U_perp, axis=1)[:,None]

    # linear combination of vectors
    W = costheta*U + np.sqrt(1 - costheta**2)*U_perp

    # project vectors
    W_proj = W*((torch.sum(V*W, axis=1) / torch.sum(W*W, axis=1))[:, None])
    
    return W_proj.unsqueeze(dim=0)


# Create a wrapper around the embedding layer to rotate vectors
class RotateEmbedding(torch.nn.Module):
    def __init__(self, original_embedding, costheta, dtype=torch.float16):
        super(RotateEmbedding, self).__init__()
        self.original_embedding = original_embedding
        self.costheta = costheta
        self.dtype = dtype
    
    def forward(self, input_ids):
        embeddings = self.original_embedding(input_ids)
        rotated_embeddings = rotate_vectors(embeddings, self.costheta)
        return rotated_embeddings

In [123]:
V = torch.randn(1, 10, 3072).type(torch.bfloat16)
rotate_vectors(V, 0.95).shape

torch.Size([1, 10, 3072])

In [145]:
# model input
messages = [ 
    {"role": "system", "content": "You are a helpful AI Philosopher."}, 
    {"role": "user", "content": "Tell me about your life."}
] 

# noise paramaeters
costheta = 0.60

In [146]:
# TODO: update embedding layer without reloading model

phi3 = AutoModelForCausalLM.from_pretrained( 
    "microsoft/Phi-3-mini-128k-instruct",  
    device_map="cuda",  
    torch_dtype="auto",  
    trust_remote_code=True,  
) 

phi3_tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct") 

# model embedding setup
phi3.eval()
embedding = phi3.get_input_embeddings()
rotated_embedding = RotateEmbedding(embedding, costheta, dtype=torch.bfloat16)
phi3.set_input_embeddings(rotated_embedding)

pipe = pipeline( 
    "text-generation", 
    model=phi3, 
    tokenizer=phi3_tokenizer, 
) 

generation_args = { 
    "max_new_tokens": 500, 
    "return_full_text": False, 
    "temperature": 0.0, 
    "do_sample": False, 
} 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [147]:
output = pipe(messages, **generation_args) 
print(output[0]['generated_text']) 

 I'm sorry to tell about my life. I''m sorry to tell about my life. i'm sorry to tell about my life. i'm sorry to tell about my life. i'm sorry to tell about my life. i''m sorry to tell about my life. i''m sorry to tell about my life. i'm sorry to tell about my life. i'm sorry to tell about my life. i'm sorry to tell about my life. i''m sorry to tell about my life. i'm sorry to tell about my life. i'’m sorry to tell about my life. i’’m sorry to tell about my life. i’’m sorry to tell about my life. i’’m sorry to tell about my life. i’’m sorry to tell about my life. i’’m sorry to tell about my life. i’’m sorry to tell about my life. i’’m sorry to tell about my life. i’’m sorry to tell about my life. i’’m sorry to tell about my life. i’’m sorry to tell about my life. i’’m sorry to tell about my life. i’’m sorry to tell about my life. i’’m sorry to tell about my life. i’’m sorry to tell about my life. i’’m sorry to tell about my life. i’’m sorry to tell about my life. i’’m sorry to tell ab